In [2]:
from IPython.core import display
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, cross_validation
from sklearn.ensemble import RandomForestRegressor
from sklearn.grid_search import GridSearchCV

%matplotlib inline

In [9]:
def readCsv(name, nrows=50000):
    df = pd.read_csv(
        "data/expedia/{}.csv".format(name), 
        nrows=nrows,
        parse_dates=["date_time"])
    df["year"] = pd.DatetimeIndex(df['date_time']).year
    df["month"] = pd.DatetimeIndex(df['date_time']).month
    df["hour"] = pd.DatetimeIndex(df['date_time']).hour
    df["dayofweek"] = pd.DatetimeIndex(df['date_time']).dayofweek
    return df

df = readCsv("train")
print("shape: ", df.shape)
df.sample(5)

('shape: ', (50000, 28))


,date_time,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,...,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,year,month,hour,dayofweek
6020,2014-12-29 15:08:05,25,2,23,48,4924,NaN,35370,0,0,...,1,1,3,106,756,5,2014,12,15,0
14020,2013-12-03 21:41:35,37,1,69,749,51392,NaN,71855,0,1,...,0,1,2,50,675,70,2013,12,21,1
15386,2013-02-07 15:53:05,2,3,66,174,21583,2482.9174,76943,0,0,...,0,2,2,50,213,92,2013,2,15,3
16970,2014-03-07 08:09:04,2,3,66,322,12237,NaN,81712,0,1,...,0,1,0,140,59,92,2014,3,8,4
1675,2013-06-10 20:42:06,2,3,66,226,9427,1140.8046,7783,0,1,...,1,1,4,47,1502,99,2013,6,20,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 28 columns):
date_time                    50000 non-null datetime64[ns]
site_name                    50000 non-null int64
posa_continent               50000 non-null int64
user_location_country        50000 non-null int64
user_location_region         50000 non-null int64
user_location_city           50000 non-null int64
orig_destination_distance    30998 non-null float64
user_id                      50000 non-null int64
is_mobile                    50000 non-null int64
is_package                   50000 non-null int64
channel                      50000 non-null int64
srch_ci                      49971 non-null object
srch_co                      49971 non-null object
srch_adults_cnt              50000 non-null int64
srch_children_cnt            50000 non-null int64
srch_rm_cnt                  50000 non-null int64
srch_destination_id          50000 non-null int64
srch_destination_type_id    

In [10]:
df.describe()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,year,month,hour,dayofweek
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,30998.000000,50000.000000,50000.000000,50000.000000,50000.000000,...,50000.000000,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,8.897180,2.650360,83.790000,314.080660,28919.517620,1865.603288,102767.353140,0.128900,0.252600,5.823500,...,0.082080,1.473040,3.266200,79.83306,612.266840,49.637080,2013.713580,6.988820,13.405460,2.739780
std,12.048772,0.745415,52.748348,210.529099,16893.807429,2052.086290,49086.616878,0.335093,0.434508,3.708658,...,0.274489,1.172903,1.665315,54.11810,513.465159,28.942927,0.452093,3.328158,5.442688,1.981389
min,2.000000,0.000000,0.000000,0.000000,3.000000,0.005600,12.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.00000,0.000000,0.000000,2013.000000,1.000000,0.000000,0.000000
25%,2.000000,3.000000,66.000000,174.000000,14458.000000,269.992450,69003.000000,0.000000,0.000000,2.000000,...,0.000000,1.000000,2.000000,50.00000,190.000000,25.000000,2013.000000,4.000000,10.000000,1.000000
50%,2.000000,3.000000,66.000000,311.000000,29370.000000,1105.370050,107548.000000,0.000000,0.000000,9.000000,...,0.000000,1.000000,2.000000,50.00000,628.000000,48.000000,2014.000000,7.000000,14.000000,3.000000
75%,11.000000,3.000000,69.000000,385.000000,43181.500000,2537.089075,144246.000000,0.000000,1.000000,9.000000,...,0.000000,2.000000,4.000000,105.00000,705.000000,73.000000,2014.000000,10.000000,18.000000,4.000000
max,53.000000,4.000000,239.000000,1025.000000,56472.000000,11641.224200,181983.000000,1.000000,1.000000,10.000000,...,1.000000,29.000000,6.000000,212.00000,2113.000000,99.000000,2014.000000,12.000000,23.000000,6.000000
